# QOSF mentorship screening task 3
Aim: To create a Quantum Circuit Simulator. 

Features of the simulator: Support any single qubit Quantum gate  
                                          Support for controlling any single qubit gate with arbitrary control and target qubits  
                                          Support parametric gates
                           
### Theory
Classically, n bits can be in one (and only one) of 2<sup>n</sup> states at any given time (since each bit can take 2 values: 0,1). Thus, n bits Classical information can be expressed with exactly n bits.

In the quantum realm however, n qubits can be in a superposition of the 2<sup>n</sup> classical states. This means that the n bit representation we used for representing classical information is insufficient for expressing quantum information. Therefore, we need a system that represents the probability of getting all 2<sup>n</sup> states on measurement.  
However, qubits have another trick up their sleeves- <b>Phase</b>

Two Qubits that produce the same result (with similar probabilities) on measurement could still be in different states. This is because of the phase in which the qubits exist.  
Phase can be of 2 types: i) Local Phase  
                                       ii) Global Phase   
Global Phase does not usually affect how operators act on the qubits, but is essential for some algorithms such as Shor's algorithm (where we try to measure the global phase to estimate the period of modular exponentiation)  
Local phase on the other hand is known to produce very intersting effects such as phase kickback, which is essential for the working of several quantum algorithms

So, we need a representation which incorporates both the probability as well as the phase of the (classical) states of the qubits. We use a vector representation called the statevector to express the present state of the system. Some important properties of statevectors are:
* Since there are 2<sup>n</sup> possible states, the statevector is also 2<sup>n</sup> dimensional.  
* Each complex entry in the statevector represents the amplitude if the state function in that state. The square of the modulus of the entry represents the probability of realizing that state on measurement. 

Amplitudes can be altered using operators. Operators are unitary matrices which operate on the statevactor to alter the state amplitudes.  

We will now explore these concepts further by creating our own Quantum Circuit Simulator!

In [14]:
import numpy as np
from numpy.random import choice
import math
import cmath

## We create the Simulator using the following functions:
* get_ground_state :: Creates a statevector in the ground state. The ground state is the state where all the qubits are in the state 0.
* run_program &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;:: Runs the program (Circuit) and calculates the final statevector
* get_unitary $\qquad$:: return unitary operator of size 2<sup>n</sup> x 2<sup>n</sup> for given gate and target qubits
* get_counts&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;:: Uses the probability obtained by finding the square of absolute values of the entries in the statevector and performs probabilistic measurement of the system.
* measure_some&nbsp;&nbsp;&nbsp;&nbsp; :: (Support function for get_counts) Uses the counts dictionary retreived from get_counts to get the measurement results only for the qubits in measured_qubits

In [20]:
def get_ground_state(num_qubits):
    # Creates a statevector in the ground state. The ground state is the state where all the qubits are in the state 0. Thus, 
    # for a 2 qubit circuit, the statevector would be [1,0,0,0]
    
    # Arguments:
    # num_qubits :: (Integer) No. of qubits in Quantum circuit
    
    statevector = np.zeros(2**num_qubits)
    statevector[0] = 1
    return statevector

def run_program(initial_state, program, params = []):
    # Runs the program (Circuit) and calculates the final statevector
    
    # Arguments:
    # initial_state :: Statevector before the program is executed. 
    # program :: Quantum circuit that is to be executed. The program can contain one of the gates as stated in dictionary 'gates'
               # below or can be the general gate u3 with arbitrary parameters theta, phi and lambda
    # params :: Parameters to be used for variational circuits. 
    
    params = list(params)
    gates = {                                              # Dictionary of u3 parameters for the gates: H, X, Y, Z, I, S, S_dagger
        'h': [np.pi/2, 0, np.pi],
        
        'x': [np.pi, 0, np.pi],
        
        'y': [np.pi, np.pi/2, np.pi/2],  
        
        's': [0, np.pi/4, np.pi/4],
        
        'sdg': [0, -np.pi/4, -np.pi/4],
        
        'z': [0, np.pi/2, np.pi/2],
        
        't': [0, np.pi/8, np.pi/8],
        
        'i': [0, 0, 0],
    }
    # Definition of the u3 gate. All the gates mentioned above execute this lambda expression using the parameters in the dictionary
    u3 = lambda t,f,l: np.array([[math.cos(t/2),                       -cmath.exp(1j*l)*math.sin(t/2)],  
                                [cmath.exp(1j*f)*math.sin(t/2),    cmath.exp(1j*(f+l))*math.cos(t/2)]])
    
    total_states = len(initial_state)                               
    num_qubits = int(math.log(total_states, 2))                     # Since total_states = 2**num_qubits, num_qubits = log(total_states)
    program_unitary = np.eye(total_states)                          # Initializing the program unitary
    params.reverse()                                                # Because we use params.pop()
    
    for operation in program:    
        if "params" not in operation.keys():
            unitary = get_unitary(num_qubits, u3(*gates[operation['gate']]), operation["target"])
        else:
            updated_params = [params.pop() if type(i) == str else i for i in operation["params"].values()]
            unitary = get_unitary(num_qubits, u3(*updated_params), operation["target"])
        program_unitary = unitary @ program_unitary
        
    final_statevector = program_unitary @ initial_state
    return final_statevector

def get_unitary(total_qubits, gate_unitary, target_qubits):
    # return unitary operator of size 2**n x 2**n for given gate and target qubits
    
    # Arguments:
    # total_qubits :: (Integer) No. of qubits in qpu
    # gate_unitary :: (2D numpy array) Gate unitary to be applied on the qubits
    # target_qubits :: (1D list) qubits on which operator is to be applied. If number of target qubits = 1: Single Qubit gate
                                                                          # Else, it is assumed to be a controlled gate (CU) 
                                                                          # and the gate unitary is U in this case
    
    apply_I = lambda u: np.kron(u, np.eye(2))
    unitary = np.array([1])
    q0 = np.array([[1,0],[0,0]])
    q1 = np.array([[0,0],[0,1]])
    
    # Single Qubit 
    if len(target_qubits) == 1:
        for i in range(total_qubits):
            if i == target_qubits[0]:
                unitary = np.kron(unitary, gate_unitary)
            else:
                unitary = apply_I(unitary)
                
    # Multi Qubit gate
    elif len(target_qubits) > 1: 
        
        #case: control bit = 0
        for i in range(total_qubits): 
            if i == target_qubits[0]:
                unitary = np.kron(unitary, q0)
            else:
                unitary = apply_I(unitary)
                
        #case: control bit = 1
        unitary2 = np.array([1])
        for i in range(total_qubits):
            if i == target_qubits[0]:
                unitary2 = np.kron(unitary2, q1)
            elif i == target_qubits[1]:
                unitary2 = np.kron(unitary2, gate_unitary)
            else:
                unitary2 = apply_I(unitary2)
                
        #adding both cases
        unitary = unitary + unitary2
    return unitary

def get_counts(state_vector, num_shots, measured_qubits = None):
    # Uses the probability obtained by finding the square of absolute values of the entries in the statevector and performs
    # probabilistic measurement of the system.
    
    # Arguments: 
    # state_vector :: Statevector of the system which is to be measured.
    # num_shots :: Number of measurements to be made.
    # measured_qubits :: Select the qubits on which the measurement is to be made
    
    measure_all = lambda a: choice(range(len(state_vector)), 1, p=np.square(np.abs(state_vector)))
    num_qubits = int(math.log(len(state_vector), 2))
    
    counts = {}
    for _ in range(num_shots):
        outcome = format(measure_all(state_vector)[0], f'0{num_qubits}b')
        if outcome not in counts.keys():
            counts[outcome] = 1
        else:
            counts[outcome] += 1
            
    if not measured_qubits == None:
        counts = measure_some(counts, measured_qubits)
    return counts

def measure_some(counts, measured_qubits):
    ## Support function for get_counts ##
    # Uses the counts dictionary retreived from get_counts to get the measurement results only for the qubits in measured_qubits
    
    # Arguments:
    # counts :: Counts returned by get_counts
    # measured_qubits :: List if qubits to be measured
    
    new_counts = {}
    for key, value in counts.items():
        new_key = ''
        for i in range(len(key)):
            if i in measured_qubits:
                new_key += key[i]
        if new_key in new_counts.keys():
            new_counts[new_key] += value
        else:
            new_counts[new_key] = value
    return new_counts
    

## Using the Simulator:
The input should be given as a list of dictionaries with each dictionary having form similar to the examples below:  

Single Qubit gate :                                        { "gate": "x", "target": [1] }  

Controlled single Qubit gate :                      { "gate": "x", "target": [0, 1] }  

Parametric gate :                                           { "gate": "u3", 'params' :{'theta': np.pi, 'phi': -np.pi/2, 'lambda': np.pi/2}, "target": [0] }  

Variational controlled parametric gate :       { "gate": "u3", 'params': {'theta': 'param1', 'phi': -np.pi/2, 'lambda': np.pi/2}, "target": [0,1] }  

The simulator supports the following single qubit gates: H, X, Y, S, S†, Z, T, I, U3  
Controlled gate (CU) for any of the gates can be created by simply specifying the 'target' as a list containing [source, target]. The gate U should be one of the single qubit gates mentioned above and should be passed as the 'gate' parameter.  
Any arbitrary values can be given for the parameters Theta, Phi and Lambda for the U3 gate. 

## Let's try our simulator out

### 1. Deutsch-Jozsa Algorithm
Problem Statement: Given a binary function which is guaranteed to be either balanced (returns 0 and 1 equal no. of times) or constant (gives 0 always or gives 1 always), idnetify which type of funtion it is. 

Classically, one would need to keep trying the funciton till you get a different answer (which means that it is a balanced function). You need to check for at least 2<sup>n</sup> + 1 inputs before you can determine if the fucntion is balanced or constant. 

The Deutsch-Jozsa Algorithm algorithm however, can determine this property about the function with just one call to the fucntion.
    
You can find more information about Deutsch-Jozsa Algorithm on the [Qiskit Textbook](https://qiskit.org/textbook/ch-algorithms/deutsch-jozsa.html) or from "Quantum Computation and Quantum Information" by Isaac Chuang and Michael Nielsen.

In [16]:
num_qubits = 5
function = 'balanced'
num_shots = 1

# Function to generate the oracle
def gen_oracle(program, function, num_qubits):
    if function == 'constant':
        program.append({ "gate": "x", "target": [num_qubits] })
    else:
        for i in range(num_qubits-1, -1, -1):
            program.append({ "gate": "x", "target": [i, num_qubits] }) 

# Convenience function to put H gates on qubits [0, num_qubits-1]
def equal_superposition(program, num_qubits):
    for i in range(num_qubits):
        program.append({ "gate": 'h', "target": [i] })

# Creating the circuit
program = []

program.append({ "gate": 'x', "target": [num_qubits] })

equal_superposition(program, num_qubits+1)

gen_oracle(program, function, 5)

equal_superposition(program, num_qubits)

# Running the program
qpu = get_ground_state(num_qubits+1)
state_vector = run_program(qpu, program)

# Performing the measurements on the statevector other than the Ancillia qubit. 
print(get_counts(state_vector, num_shots, list(range(5))))

{'11111': 1}


The ouput being '11111' implies that the function is balanced and a result of '00000' implies that the function is constant

## 2. Finding the minimum Eigenvalue using VQE
(QOSF task 4 of the previous cohort)  

Find the min. eigenvalue of the hamiltonian:
$$\begin{bmatrix} 1&0&0&0 \\ 0&0&-1&0 \\ 0&-1&0&0 \\ 0&0&0&1  \end{bmatrix}$$

This matrix can be decomposed into the following combination of operators: 1/2(i⊗i + z⊗z) - 1/2(x⊗x + y⊗y)

We use VQE to find the eigenvalues for all the component hamiltonians (i⊗i, z⊗z, x⊗x and y⊗y) and put them into the equation above to find the eigenvalue of the above hamiltonian. We use this as the objective function and try to minimize it.

You can find my submission (from the previous cohort) for this problem [here](https://github.com/anshsingal/qosf_task_4/blob/master/VQE_final.ipynb).

In [17]:
# Eigenvalues for the gates II,zz, XX, YY corresponding to their respective measurement outcomes. 
eig_values = [{'00':1, '10':1, '01':1, '11':1},
              {'00':1, '10':-1, '01':-1, '11':1},
              {'00':1, '10':-1, '01':-1, '11':1},
              {'00':1, '10':-1, '01':-1, '11':1}]

# Lambda expression to find the eigenvalue of the hamiltonian
calculate_expectation = lambda a: 1/2*(a[0] + a[1]) - 1/2*(a[2] + a[3])

num_shots = 100

# Creating 4 variational circuits for performing VQE on all 4 component circuits.
circuit = []
for i in range(4):
    circuit.append([
        { "gate": "h", "target": [0] },
        { "gate": "i", "target": [1] },
        { "gate": "x", "target": [0,1] },
        { "gate": "u3", 'params': {'theta': 'param1', 'phi': -np.pi/2, 'lambda': np.pi/2}, "target": [0] },
        { "gate": "i", "target": [1] }
    ])
    
# Adding provisions for measurement
circuit[2].append({ "gate": "h", "target": [0] })
circuit[2].append({ "gate": "h", "target": [1] })

circuit[3].append({ "gate": "sdg", "target": [0] })
circuit[3].append({ "gate": "sdg", "target": [1] })
circuit[3].append({ "gate": "h", "target": [0] })
circuit[3].append({ "gate": "h", "target": [1] })

qpu = get_ground_state(2)

In [18]:
def get_cost(params):
    # Returns the objective function (eigenvalue) for the given value of ansatz (params)
    
    # Arguments: 
    # params :: Value of ansatz used
    
    counts = [0 for i in range(4)]
    
    for i in range(4):
        new_state = run_program(qpu, circuit[i], params.copy())
        counts[i] = get_counts(new_state, num_shots)
    
    min_eigen = 100
    out2 = []
    
    # Calculating the expectation value of the eigenvalue for the given value of ansatz. 
    for j in range(4):
        avg = 0
        for key, value in counts[j].items():
            avg += eig_values[j][key]*value
        out2.append(avg/num_shots)
        
    return calculate_expectation(out2)

In [19]:
# Running the optimizer to get the minimum eigenvalue
from scipy.optimize import minimize

initial_params = [0]
boundary = [(0,2*np.pi)]
minimize(get_cost, initial_params, method = 'Powell', bounds = boundary)

   direc: array([[1.]])
     fun: array(-1.)
 message: 'Optimization terminated successfully.'
    nfev: 38
     nit: 2
  status: 0
 success: True
       x: array([3.15783208])

We see that the Minimum eigenvalue has a value -1 at value of ansatz ≈ π (for domain = [0,2π])